In [8]:
# IMPORTS
from VGG_Pretrain import *
from Build_Histogram import *
from Load_Dataset_Folder import *
from Features_Processing import *
from Linear_Processsing_Pipeline import *
from Training_Poly_Processing_Pipeline import *
from Testing_Poly_Processing_Pipeline import *
from Evaluate_Train_Accuracy_of_Model import *
from LeNet_Implementation import *
from torch import optim
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import train_test_split
import torch.utils.data as data
import torchvision.models as models

In [9]:
user= 'Max'
root_path = "C:\\Users\\" + user + "\\Documents\\GitHub\\Shark-Species-Classification"
data_path = os.path.join(root_path, 'Genus Carcharhinus')

## SVC & KMeans

In [10]:

image_files, labels = load_dataset_folder(data_path)
features, processed_labels = Features_Processing(image_files, labels)


features_train, features_test, labels_train, labels_test = train_test_split(features, processed_labels, test_size = .2, random_state = 0)
features_train_train, features_validation, labels_train_train, labels_validation = train_test_split(features_train, labels_train, test_size = .25, random_state = 0)


c, d = Poly_Processing_Pipeline(features_train_train, features_validation, labels_train_train, labels_validation)


training_accuracy = Testing_Poly_SVC(features_train_train, features_validation, labels_train_train, labels_validation, c, d)
testing_accuracy = Testing_Poly_SVC(features_train, features_test, labels_train, labels_test,c, d)

print("c: ", c)
print("d: ", d)
print("Training Accuracy: ", training_accuracy)
print("Testing Accuracy: ", testing_accuracy)


c:  0.1
d:  4
Training Accuracy:  0.7333333333333333
Testing Accuracy:  0.6086956521739131


## VGG Pretrained

In [19]:
dataset = ImageLoader(data_path)

train_dataset, test_dataset = Dataset_Splitter(.5, dataset)
train_train_dataset, validation_dataset = Dataset_Splitter(.9, train_dataset)

In [20]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_train_dataset,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(validation_dataset,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_dataset,
                                batch_size=BATCH_SIZE)

In [21]:
vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
vgg11_layers = get_vgg_layers(vgg11_config, batch_norm=True)

OUTPUT_DIM = 9
model = VGG(vgg11_layers, OUTPUT_DIM)

In [22]:
pretrained_model = models.vgg11_bn(pretrained=True)

IN_FEATURES = pretrained_model.classifier[-1].in_features
final_fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)

pretrained_model.classifier[-1] = final_fc

In [23]:
model.load_state_dict(pretrained_model.state_dict())

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 128,808,713 trainable parameters


In [24]:
LR = 1e-3
params = [
          {'params': model.features.parameters(), 'lr': LR / 10},
          {'params': model.classifier.parameters()}
         ]

optimizer = optim.Adam(params, lr=LR)

device = torch.device('cpu')

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [25]:

EPOCHS = 7

best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):


    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'VGG-model_2.pt')


    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epochs:   0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

	Train Loss: 2.147 | Train Acc: 24.81%
	 Val. Loss: 1.775 |  Val. Acc: 41.67%


Training:   0%|          | 0/2 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

	Train Loss: 0.071 | Train Acc: 97.87%
	 Val. Loss: 2.565 |  Val. Acc: 50.00%


Training:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

	Train Loss: 0.001 | Train Acc: 100.00%
	 Val. Loss: 1.720 |  Val. Acc: 58.33%


In [29]:
model.load_state_dict(torch.load('VGG-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Test Loss: 0.745 | Test Acc: 85.11%


## Custom CNN

In [ ]:
OUTPUT_DIM = 9
model = AlexNet(OUTPUT_DIM)

In [13]:
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda') #if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

EPOCHS = [1,3,5,10,15,20,30]
EPOCHS = 7

best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'MMnet-model.pt')

    print(f'Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

NameError: name 'model' is not defined